## Instalando os pacotes

In [26]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [27]:
import pandas as pd
import glob
import os

## Lendo os arquivos

In [57]:
df = pd.read_csv(arquivo, sep=",", skiprows=len(metadados) + 1)
print(df.columns)

Index(['DATETIME', 'TEMPERATURA_MEDIA'], dtype='object')


In [60]:
# Caminho da pasta onde estão os arquivos
pasta = r"D:\Documentos\Doutorado\Python\python_env_app\dados\Dados_Sul"

# Lendo os arquivos csv
arquivos = glob.glob(os.path.join(pasta, "*.csv"))

# Lista para armazenar os dataframes
lista_dfs = []

for arquivo in arquivos:
    # ------------------------------------------------------------------
    # 1. EXTRAÇÃO DOS METADADOS E CONTAGEM DAS LINHAS A SEREM PULADAS
    # ------------------------------------------------------------------
    metadados = {}
    linhas_puladas = 0
    with open(arquivo, "r", encoding="utf-8") as f:
        for line in f:
            linhas_puladas += 1
            line = line.strip()
            
            # Condição de parada: Linha vazia ou linha de cabeçalho
            if line == "" or line.startswith("DATETIME") or line.startswith("DATA"): 
                break
                
            if ":" in line:
                chave, valor = line.split(":", 1)
                metadados[chave.strip()] = valor.strip()
    
    # ------------------------------------------------------------------
    # 2. LEITURA DA TABELA DE DADOS COM PANDAS
    # ------------------------------------------------------------------
    
    # Usamos linhas_puladas que agora deve apontar para o cabeçalho (ou a linha acima dele)
    # se a sua lógica original de "len(metadados) + 1" estava correta, mantenha ela
    # se a nova lógica de quebra de linha funcionar, use linhas_puladas
    df = pd.read_csv(arquivo, sep=",", skiprows=linhas_puladas - 1) 
    
    # ------------------------------------------------------------------
    # 3. PRÉ-PROCESSAMENTO DO DATAFRAME
    # ------------------------------------------------------------------

    # Limpa espaços em branco dos nomes das colunas (Ex: ' DATETIME ' vira 'DATETIME')
    df.columns = df.columns.str.strip()
    
    # Colunas com os nomes corretos (para visualização)
    print(f"Colunas do arquivo {os.path.basename(arquivo)}:")
    print(df.columns)
    
    # Conversão para DATETIME e padronização do nome da coluna
    if 'DATETIME' in df.columns:
        df['DATETIME'] = pd.to_datetime(df['DATETIME'], dayfirst=True)
    elif 'DATA' in df.columns:
        # Renomeia para 'DATETIME' antes de converter, para padronizar
        df = df.rename(columns={'DATA': 'DATETIME'})
        df['DATETIME'] = pd.to_datetime(df['DATETIME'], dayfirst=True)
        
    # Adiciona colunas com informações da estação (metadados)
    for chave, valor in metadados.items():
        # Limpa o nome da coluna de metadados para evitar espaços
        df[chave.replace(" ", "_")] = valor 
        
    # Adiciona à lista de dataframes
    lista_dfs.append(df)

# Junta tudo em um só dataframe
df_todas = pd.concat(lista_dfs, ignore_index=True)

# Mostra um resumo
print("\n" + "="*50)
print("Dataframe Consolidado (df_todas)")
print("="*50)
print(df_todas.info())
print("\nHead do Dataframe:")
print(df_todas.head())

Colunas do arquivo dados_CWB_D_2020-01-01_2024-12-31.csv:
Index(['DATETIME', 'TEMPERATURA_MEDIA'], dtype='object')


ValueError: time data "1/13/2020" doesn't match format "%d/%m/%Y", at position 12. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [48]:
print("Nomes das colunas em df_todas:")
print(df_todas.columns.tolist())

Nomes das colunas em df_todas:
['DATETIME,TEMPERATURA_MEDIA', 'Nome', 'Codigo Estacao', 'Latitude', 'Longitude', 'Altitude', 'Situacao', 'Data Inicial', 'Data Final', 'Periodicidade da Medicao']


In [56]:
# 1. Defina a lista de colunas desejadas (correto)
colunas_desejadas = ['DATETIME,TEMPERATURA_MEDIA', 'Nome', 'Codigo Estacao', 'Latitude', 'Longitude', 'Altitude']
df_novo = df_todas[colunas_desejadas]